In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# BASIC SETUP

In [ ]:
# BASIC SETUP
! [ ! -z "$COLAB_GPU" ] && pip install torch skorch && pip install neptune-client

!cp "drive/My Drive/dl_project_data/repo/data_loading.py" .
!mkdir ./helper_scripts/
!cp "drive/My Drive/dl_project_data/repo/helper_scripts/visual_helpers.py" ./helper_scripts
!cp "drive/My Drive/dl_project_data/repo/architecture.py" .
!cp "drive/My Drive/dl_project_data/repo/model_training.py" .
!mkdir ./train/
# Creates RAM-Disk for potential speed-up
!sudo mount -t tmpfs -o size=7g tmpfs train
#!for i in 0; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;
!for i in 0 1 2 3 4 5 6 7 8 9 10 11; do cp "drive/My Drive/dl_project_data/train/$i.tar" ./train/; tar -xf "./train/$i.tar" -C ./train/; rm "./train/$i.tar"; done;

     |████████████████████████████████| 122kB 3.1MB/s 
     |████████████████████████████████| 92kB 2.9MB/s 
     |████████████████████████████████| 829kB 10.7MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 204kB 15.4MB/s 
     |████████████████████████████████| 163kB 16.1MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 133kB 15.9MB/s 
     |████████████████████████████████| 143kB 12.7MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.118-py2.py3-none-any.whl size=149617 sha256=1d4bae6edb248a0de601d7dafca8a632ff981d5869b213a66b936b014f2fd471
  Stored in directory: /root/.cache/pip/wheels/f2/0e/03/b7fcb3255a708609c21d8243cf6ba4f0419a625474e3c9c01c
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b83b1cc919051cf5d0dc49a80cefb1920e457c4c68473f42221adda8133ff29d
  Stored in dire

# IMPORTS
**You should not have to change anything here.**

In [ ]:
# IMPORTS
from collections import OrderedDict

import torch
from torch import nn
from torchvision import models, transforms

import skorch.callbacks as scb
from skorch import NeuralNetBinaryClassifier

import model_training as md
import architecture as arch
from data_loading import ToTensor, Normalize, RandomRotation, RandomHorizontalFlip

# Skorch uses some depricated scikit code - works just fine, so surpress warnings for clean output
import warnings
warnings.filterwarnings('ignore')

# CLASSIFIER PARAMETRIZATION

Here you can parametrize your model and set loss, optimizer, learning rate, etc. 

For further information on what can be set and how, please refer to the [skorch documentation](https://skorch.readthedocs.io/en/stable/classifier.html#skorch.classifier.NeuralNetClassifier).

In [ ]:
# CLASSIFIER PARAMETRIZATION
classifier = NeuralNetBinaryClassifier(
    arch.DenseNet201,
    optimizer = torch.optim.Adam, 
    max_epochs = 30,
    lr = 0.01,
    batch_size = 128,
    iterator_train__shuffle = True, # Shuffle training data on each epoch
    train_split = None,
    callbacks = [scb.LRScheduler(policy = 'ExponentialLR', gamma = 0.9)], 
    device ='cuda')

# CLASSIFIER TRAINING

After you have added the shared folder with the data to your drive as a shortcut, you should not have to change anything here. At least for now.

**IF YOU WANT TO TRAIN WITH THE FULL DATASET, JUST REMOVE** *_small* **FROM THE CSV FILE.** 

In [ ]:
# CLASSIFIER TRAINING
md.train_model(classifier, 
            train_labels = "drive/My Drive/dl_project_data/train/train_split.csv", 
            test_labels = "drive/My Drive/dl_project_data/train/test_split.csv", 
            file_dir = "train", 
            train_transform = transforms.Compose([transforms.ToPILImage(),
                                  #transforms.Pad(64, padding_mode='reflect'), # 96 + 2*64 = 224
                                  transforms.RandomHorizontalFlip(),  # TODO: model expects normalized channel values (substract means)
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20),
                                  transforms.ToTensor()]),
            test_transform = transforms.ToTensor(),
            in_memory = False,
            output_path = ".",
            #output_path = "drive/My Drive/dl_project_data/output",
            logger = {
                "api_token": "",
                "project_qualified_name": "elangenhan/hcd-experiments",
                "experiment_name": "DenseNet201"
            }
)

https://ui.neptune.ai/elangenhan/hcd-experiments/e/HCDEX-4
Starting Training for <class 'architecture.DenseNet201'> 
          Model-Params:
              Criterion:     <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>
              Optimizer:     <class 'torch.optim.adam.Adam'>
              Learning Rate: 0.01
              Epochs:        2
              Batch size:    128
              


  epoch    test_acc    test_f1    test_precision    test_recall    test_roc_auc    train_acc    train_f1    train_loss    train_precision    train_recall    train_roc_auc       dur
-------  ----------  ---------  ----------------  -------------  --------------  -----------  ----------  ------------  -----------------  --------------  ---------------  --------
      1      0.6117     0.1151            0.8487         0.0617          0.5271       0.7682      0.7041        0.5881             0.7194          0.6894           0.8316  146.7506


      2      0.6694     0.3383            0.9337         0.2066          0.5982       0.8274      0.7770        0.4028             0.8039          0.7519           0.8945  147.5879


Saving model...
